# Section 0

QPM: Assignment 3

DENG Yunyun <br>
LI Jiaxin <br>
SBAI Ilyas <br>
ZHOU Zhichen <br>

Note: NA

# Section 1

We retrieve a dataframe of the list of the S&P500 consistuents from Wikipedia and store it in ```tic```. We then replace any '.' in the tickers' names by a '-' to match the format required by yfinance. We finally turn this dataframe into a unique list of tickers that we use to retrive the stocks data using yfinance.

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

# Date range
start_date = "2000-01-01"
end_date = "2022-12-31"

# We retrieve the list of tickers of the S&P500 from Wikipedia
URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tic = pd.read_html(URL)[0]['Symbol']
tic = tic.str.replace('.', '-', regex=True) # using regex=True to suppress a warning related to a future change in pandas's feature
tic = tic.unique().tolist()

# Importing the stock data using the download function from the yf package
data = (yf.download(tickers=tic,
start=start_date,
end=end_date)
.reset_index()
.rename(columns = {"Date": "date",
                   "Open": "open",
"High": "high",
"Low": "low",
"Close": "close",
"Adj Close": "adjusted",
"Volume": "volume"
})
)
data.head()

[*********************100%%**********************]  503 of 503 completed


8 Failed downloads:
['BALL', 'GIS', 'LDOS', 'AKAM', 'DOV', 'DOW']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2000-01-01 -> 2022-12-31)')
['VLTO', 'KVUE']: Exception("%ticker%: Data doesn't exist for startDate = 946702800, endDate = 1672462800")


date   adjusted                                                  \
                      A AAL      AAPL ABBV ABNB       ABT      ACGL ACN   
0 2000-01-03  43.757732 NaN  0.848323  NaN  NaN  9.081111  1.277778 NaN   
1 2000-01-04  40.415131 NaN  0.776801  NaN  NaN  8.821650  1.270833 NaN   
2 2000-01-05  37.908180 NaN  0.788168  NaN  NaN  8.805433  1.388889 NaN   
3 2000-01-06  36.464790 NaN  0.719961  NaN  NaN  9.113541  1.375000 NaN   
4 2000-01-07  39.503517 NaN  0.754065  NaN  NaN  9.210839  1.451389 NaN   

              ... volume                                               \
        ADBE  ...   WYNN      XEL       XOM     XRAY XYL      YUM ZBH   
0  16.274670  ...    NaN  2738600  13458200   582300 NaN  3033493 NaN   
1  14.909399  ...    NaN   425200  14510800   317700 NaN  3315031 NaN   
2  15.204176  ...    NaN   500200  17485000  1188000 NaN  4642602 NaN   
3  15.328289  ...    NaN   344100  19461600   534300 NaN  3947658 NaN   
4  16.072985  ...    NaN   469500  16603800  1401000 NaN  6063647 NaN   

                         
      ZBRA     ZION ZTS  
0  1055700  1199600 NaN  
1   522450   816100 NaN  
2   612225  1124700 NaN  
3   263925  1112100 NaN  
4   333900   782000 NaN  

[5 rows x 3019 columns]

We drop the columns that contain only the NaN value and keep only the company names that have less than 100 missing observations. 

In [2]:
data = data.dropna(axis=1, how = 'all')
data = data.loc[:, (data.isna().sum(axis=0) <= 100)]
prices = data['adjusted']
prices.head()

,A,AAPL,ABT,ACGL,ADBE,ADI,ADM,ADP,ADSK,AEE,...,WMT,WRB,WST,WY,XEL,XOM,XRAY,YUM,ZBRA,ZION
0,43.757732,0.848323,9.081111,1.277778,16.274670,28.706917,6.514027,25.075775,8.052905,10.858142,...,43.717701,1.238296,5.431206,11.691637,7.112460,18.668200,6.695475,4.725970,25.027779,36.586044
1,40.415131,0.776801,8.821650,1.270833,14.909399,27.254663,6.446171,25.075775,7.660816,10.858142,...,42.081867,1.200195,5.487315,11.252419,7.276232,18.310631,6.695475,4.630976,24.666668,34.814415
2,37.908180,0.788168,8.805433,1.388889,15.204176,27.652536,6.344389,24.834656,7.178246,11.278185,...,41.223057,1.181144,5.464872,11.848503,7.556987,19.308853,6.811220,4.654723,25.138889,34.773212
3,36.464790,0.719961,9.113541,1.375000,15.328289,26.896559,6.378318,25.166182,6.740916,11.236179,...,41.672897,1.249726,5.408763,12.402757,7.486797,20.307062,6.820126,4.615143,23.777779,35.267624
4,39.503517,0.754065,9.210839,1.451389,16.072985,27.652536,6.480102,25.738842,7.540174,11.404197,...,44.821892,1.230675,5.464872,11.994911,7.486797,20.247475,6.802317,4.512232,23.513889,35.350018


We save the data in a database for future reference.

In [3]:
import sqlite3

# We only keep the columns that have some values and we stack the data because the 
# column limit is 2000
data = data.dropna(axis=1, how = 'all')
data = data.stack()

SnP500 = sqlite3.connect(
  database="/Users/ilyas/Library/CloudStorage/OneDrive-Personnel/EDHEC/M2/Quantitative Portfolio Management/snp500.sqlite"
)

(data
  .to_sql(name="S&P500 daily", 
          con=SnP500, 
          if_exists="replace",
          index = False)
)

2100457

# Section 2

We compute the log returns for all the stocks by taking the natural logarithm of the ratio between the last price and the first price.<br>

In [4]:
log_ret = pd.DataFrame(np.log(prices.iloc[-1]/prices.iloc[0])).T
log_ret

,A,AAPL,ABT,ACGL,ADBE,ADI,ADM,ADP,ADSK,AEE,...,WMT,WRB,WST,WY,XEL,XOM,XRAY,YUM,ZBRA,ZION
0,1.224081,5.027233,2.472565,3.894514,3.029077,1.728748,2.640323,2.237545,3.14438,2.079681,...,1.165164,4.05113,3.766576,0.930021,2.262965,1.751913,1.547917,3.28566,2.326791,0.25866


# Section 3

We compute the returns for all stocks for each day as the percentage change from the previous day. <br>
For these returns, we then compute the mean and volatility that we annualise by multiplying respectively by 252 and sqrt(252). <br>
We finally compute the Sharpe ratio as the ratio between the annualised mean and the annualised volatility of returns.

In [5]:
ret = pd.DataFrame(prices.pct_change())
annual_ret = (pd.DataFrame(ret.mean()).T)*252
annual_vol = (pd.DataFrame(np.sqrt(ret.var())).T)*np.sqrt(252)
Sharpe_ratio = pd.DataFrame(annual_ret.sub(0.03).divide(annual_vol))
Sharpe_ratio


,A,AAPL,ABT,ACGL,ADBE,ADI,ADM,ADP,ADSK,AEE,...,WMT,WRB,WST,WY,XEL,XOM,XRAY,YUM,ZBRA,ZION
0,0.262212,0.681875,0.440888,0.652109,0.452909,0.31547,0.430747,0.393565,0.465204,0.385733,...,0.205988,0.677416,0.606118,0.20132,0.395611,0.306854,0.273834,0.537104,0.377925,0.179681


# Section 4
It would **not** make sense to choose weights based only on the stocks' Sharpe ratio because this metric ignores the correlation between stocks. Another limitation of our computation of the Sharpe ratio is that we assumed returns to be IID when computing the annualised returns and volatility (by using the multipliers 252 and sqrt(252)), which is too strong of an assumption